# AWS API Gateway

> **Amazon API Gateway** is an AWS service allows the creation, maintenance and securing of scalable **REST**, **HTTP** and **Websocket** APIs. APIs can be created to access AWS services or other web services and data stored in the AWS Cloud. As a developer you can use the service to create APIs which serve your own clients applications or make APIs avaliable to third party app developers.

### Types of API

* **REST API** is the de facto standard for client-server communication. Although REST APIs have high latency than HTTP APIs they have full endpoint support, supporting Edge-optimised, regional and private endpoints. They also support endpoint caching and response validation unlike HTTP APIs.<br></br>
 
* **HTTP APIs** were created to be a lower latency and cost implementation of REST APIs. Amazon created the API in 2019 and report it can offer up to a 60% reduction in latency. Though it doesn't offer as many feature as REST APIs and only has regional endpoint support on AWS.<br></br>
 
* **Websocket APIs** allow you full duplex communication between client and server, they can be used to create real time applications such as financial tickers or real time chat bots.

The full breakdown by AWS on the differences between HTTP APIs and REST APIs can be found [here](https://docs.aws.amazon.com/apigateway/latest/developerguide/http-api-vs-rest.html).



## Create your first AWS Gateway REST API

Upon opening AWS API Gateway from the AWS Services dashboard you will be meet with the landing page. Here you can select the type of API you want to build (HTTP, REST or WebSocket) or import your already developed API externally. For this example we will be building a brand new REST API select build on the REST API option to create a new REST API. The goal in mind is to create REST API that integrates with a Lambda function serving a machine learning models predictions.

From the API gateway dashboard dashboard select REST API, the New API option and name the API **ML_Model** and give it a description. You will then have to select the Endpoint Type for the API.

### API Endpoint Types

* **Regional:** Creates an API which is only avaliable in the specific AWS region that the API was created.
* **Edge Optimised:** An API which is deployed to AWS Cloudfront.
* **Private:** An API which is only avaliable through a configured VPC endpoint.

More details on API endpoints can be found in the AWS documentation [here](https://docs.aws.amazon.com/apigateway/latest/developerguide/api-gateway-api-endpoint-types.html).

In our case a regional API will suffice, once selected create the API with the Create API button.


<img src="images/API_create.png?modified=12345678" />




After creating your API you will be meet with the main dashboard where you can then begin to build the API the sidebar on the left handside will give you many options to begin creating your API. The main areas we will be interested in as are follows:

* **Resources:** Collection of resources and methods such as HTTP endpoints, Lambda function or AWS services which your API can interact with.
* **Stages:** Create staging areas from your gateway like pre-deployment and post-deployment so your gateway can be tested before being served to your end user.
* **Authorizers:** Use a Lambda function or Amazon cognito user pool to control which users have access to your gateway.
* **Models:** Defines the structure of the incoming payload between cilent and server.
* **Gateway Responses:** Configure the common Gateway responses access denied, default 4XX, bad request body in the case of the gateway failing.
* **Resource Policy:** Configure access control to the API with IAM conditions or VPC endpoints.
* **Documentation:** Add documenation so that users of your API understand how it is used.<br></br>
  


<img src="images/API_sidebar.png?modified=12345678" />

Now that we have a general overview of the various options API gateway provides us to being building an API let's begin creating an API gateway which returns results from a Lambda function. 

## Integrating your first Lambda function with API Gateway

To get used to using the API gateway interface and developing new methods let's create a simple Lambda function and integrate it with our gateway. Head over to AWs Lambda and create a new Lambda function **API_test** and add the following code to the function.


In [ ]:
import json

def lambda_handler(event, context):
    # TODO implement
    return {
        'statusCode': 200,
        'body': json.dumps('Our first API gateway test was a success!!')
    }


Now head on back to API and we will create a method to test the integration with our Lambda function, head to the API you created and select resources then click the Actions drop down list. You will be presented with the various options for creating resources and methods.

* **Create Method:** Methods are the same methods your would find in any other REST API such as GET, POST, PUSH etc.
* **Create Resource:** Defines a new API resource that we can assign methods to operate on. 
* **Enable Cross-Origin Resource Sharing(CORS):** CORS sharing allows you to define how resouces in another domain might interact securely with your API Gateway more details can be found [here](https://docs.aws.amazon.com/AmazonS3/latest/userguide/cors.html). <br></br>


<img src="images/API_methods_resources.png?modified=12345678" />

Let's create a new method to get a response from our Lambda function, select actions create method and select **GET** from the dropdown. To use a method you need to "integrate" it with an endpoint on the backend these endpoints can be a number of integrations such as at HTTP endpoint, Lambda function or AWS service. Below are the type of integrations avaliable to us.

* **Lambda function:** Integrate your service with a AWS Lambda function.
* **HTTP:** Lets your API communicate with an HTTP endpoint backend.
* **Mock:** Allows the API to return a response without sending the backend, useful for testing your API.
* **AWS Service:** Integrate your API with an AWS service.
* **VPC Link:** Integrate your API with a VPC link allowing your to connect your API to private resources.

Since we are interacting with a Lambda funtion we will choose Lambda function to apply to our **GET** method, further detailed information on the types of integration can be found in the AWS docs [here](https://docs.aws.amazon.com/apigateway/latest/developerguide/api-gateway-api-integration-types.html).

### Proxy Integrations

When creating your integration you also have the option of creating it as a proxy integration. Proxy integrations allow the selected integration access to many resources and features at once without the need to specify multiple resources paths with the use of the greedy parameter **{proxy+}**. For instance if your API have the resource paths.

> - **car/{type}/{subtype}/{parts}**
> - **car/{type}/{subtype}/{colour}**
> - **car/{type}/{subtype}/{colour}/{cost}**

As your API grows it can be very tedious to specify these paths individually for your integration when you want it to have access to all avaliable resources. By creating a proxy resource with the **{proxy+}** parameter and the **ANY** method you can allow your integration to have access to all avaliable resources. A more detailed explanation of proxy integrations can be found [here](https://docs.aws.amazon.com/apigateway/latest/developerguide/set-up-lambda-proxy-integrations.html). For our Lambda function we will not currently define it as a proxy integration and finish creating our function.

<img src="images/API_method_setup.png?modified=12345678" />

After finishing creating our integration and and configuring it with the **GET** method you will be met with the Method Execution screen showing the order of the methods execution and allowing you to test the method. There four main steps when processing the request and response, in our case two request steps before the Lambda function is called and two response steps afterwards. 

* **Method Request:** This step is used to configure security settings(API keys, authorisation, validation) and query string parameters and requests headers.
* **Integration Request:** This is the step the gateway makes a request to the integration, passing along the request data and transforming it if nessecary.
* **Integration Response:** The integration response step is the result returned from the backend in our case the Lambda function. The data is then transformed if required and sent to the method response. 
* **Method Response:** The method response contains the output of the API which will be served to the user. Usually the output contains the HTTP status code, headers and a body.

You can read more about response and request data mapping [here](https://docs.aws.amazon.com/apigateway/latest/developerguide/request-response-data-mappings.html). Click the test button and then the following test button on the next screen and you will get the response from the method.


<img src="images/API_gateway_method_execution.png?modified=12345678" />

After hitting the test button you will be presented with the test pane which will display the results of initiating your method. In this case we can see the expected result, the method returned a status 200 meaning it was a succees and the a body of text from our Lambda function which we expected. Notice the response also includes the **Response Header**, API headers contain addtional metadata about the API call. They can contain information about authorisation, cookies, caching and are a good place to look for more insight into a failing method call. 

<img src="images/API_test_pane.png?modified=12345678" />

# Getting a repsonse from the API in Python

Now that we have created our API we can use the Python requests library to test our API for a response. Firstly we will need to deploy the API so that we can request a response from it. 

## Summary

* We have learned about the different types of APIs avaliable to us from API Gateway.
* We have learned about the different types of methods we can create and the integrations avaliable to us from API Gateway. 
* We now know what the difference between responses and requests and the difference between methods and resources.
* We learned how to configure the different types of methods of the API and what each of the methods do. 
* Configured an AWS Lambda function to integrate with the API and tested the method to return our API request.
